# Log likelihood calculation

$L(\theta) = -2\theta T + \int_0^T ((\alpha_s)_+ - (\alpha_s)_-)ds + \sum_{i=1}^{m+} log [(\alpha_{t_i}^{+ -})_+ + \theta] + \sum_{i=1}^{m+} log [(\alpha_{t_i}^{- -})_- + \theta]$

$\alpha_{t^-} = \eta^+ \sum_{i=1}^{n+}e^{-k(t-\tau_i^{0+})}1{t>\tau_i^{0+}} - \sum_{i=1}^{n-}e^{-k(t-\tau_i^{0-})}1{t>\tau_i^{0-}}$

https://docs.scipy.org/doc/scipy/tutorial/optimize.html

In [2]:
import numpy as np
from scipy.optimize import minimize
def rosen(x):
    """The Rosenbrock function"""
    return sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0)
x0 = np.array([1.3, 0.7, 0.8, 1.9, 1.2])  # the 4 parameters to optimize
res = minimize(rosen, x0, method='nelder-mead',
               options={'xatol': 1e-8, 'disp': True})

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 339
         Function evaluations: 571


In [3]:
res


       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: 4.861153433422115e-17
             x: [ 1.000e+00  1.000e+00  1.000e+00  1.000e+00  1.000e+00]
           nit: 339
          nfev: 571
 final_simplex: (array([[ 1.000e+00,  1.000e+00, ...,  1.000e+00,
                         1.000e+00],
                       [ 1.000e+00,  1.000e+00, ...,  1.000e+00,
                         1.000e+00],
                       ...,
                       [ 1.000e+00,  1.000e+00, ...,  1.000e+00,
                         1.000e+00],
                       [ 1.000e+00,  1.000e+00, ...,  1.000e+00,
                         1.000e+00]]), array([ 4.861e-17,  7.652e-17,  8.114e-17,  8.633e-17,
                        8.641e-17,  2.179e-16]))

In [4]:
import numpy as np
from scipy.optimize import minimize
from likelihood import MaximumLikelihood

T = 0.001
tau_0_plus = np.array([[0.1, 0.3, 0.5, 0.7, 0.9]]) * T
tau_0_minus = np.array([[0.2, 0.4, 0.6]]) * T
t_plus = np.array([[0.11, 0.31, 0.51, 0.71, 0.91]]) * T
t_minus = np.array([[0.21, 0.41, 0.61]]) * T

m = MaximumLikelihood(T, tau_0_plus, tau_0_minus, t_plus, t_minus)

x0 = np.array([1.3, 0.7, 0.8, 1.9])  # the 4 parameters to optimize
res = minimize(m.likelihood_to_minimize, x0, method='nelder-mead',
               options={'xatol': 1e-8, 'disp': True})
res

/var/folders/rr/nsd1p9hx6hs_81m8swcypvkc0000gn/T/ipykernel_52919/1025083622.py:14: RuntimeWarning: Maximum number of function evaluations has been exceeded.
  res = minimize(m.likelihood_to_minimize, x0, method='nelder-mead',


       message: Maximum number of function evaluations has been exceeded.
       success: False
        status: 1
           fun: -9.762244064601471e+24
             x: [ 1.288e+02  7.611e+27 -1.485e+28  1.925e+28]
           nit: 476
          nfev: 800
 final_simplex: (array([[ 1.288e+02,  7.611e+27, -1.485e+28,  1.925e+28],
                       [ 1.471e+02,  5.337e+27, -1.041e+28,  1.350e+28],
                       ...,
                       [ 1.745e+02,  3.856e+27, -7.524e+27,  9.753e+27],
                       [ 1.874e+02,  2.957e+27, -5.770e+27,  7.479e+27]]), array([-9.762e+24, -6.637e+24, -5.238e+24, -4.572e+24,
                       -3.426e+24]))

## Calculating integral of alpha_s

In [5]:
T=10
tau_0_plus = np.array([[0.1, 0.3, 0.5,0.7,0.9]]) * T
tau_0_minus = np.array([[0.2, 0.4, 0.6]])*T
tau_0 = np.concatenate(
    [np.zeros([1,1]), tau_0_minus, tau_0_plus, np.ones([1,1]) * T], axis=1
)

eta_plus = 100
eta_minus = 90
k = 200
eta_minus_vector = -np.ones([tau_0_minus.shape[1], 1]) * eta_minus
eta_plus_vector = np.ones([tau_0_plus.shape[1], 1]) * eta_plus
eta_vector = np.concatenate(
    [np.zeros([1,1]), eta_minus_vector, eta_plus_vector, np.zeros([1,1])], axis=0
).T
tau_eta = np.concatenate([tau_0, eta_vector])
result = eta_vector.T * np.ones([1, tau_0.shape[1]])

#tau_0.sort()
tau_eta = tau_eta[:,tau_eta[0, :].argsort()]
tau_0 = tau_eta[:,tau_eta[0, :].argsort()][0:1,:]
eta_0 = tau_eta[:,tau_eta[0, :].argsort()][1:2,:]
#print(result.T)
print(tau_eta)


[[  0.   1.   2.   3.   4.   5.   6.   7.   9.  10.]
 [  0. 100. -90. 100. -90. 100. -90. 100. 100.   0.]]


In [6]:
np.exp(-k*tau_0_plus)
tau_matrix = tau_0 * np.ones([1, tau_0.shape[1]]).T
tau_matrix

array([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  9., 10.],
       [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  9., 10.],
       [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  9., 10.],
       [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  9., 10.],
       [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  9., 10.],
       [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  9., 10.],
       [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  9., 10.],
       [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  9., 10.],
       [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  9., 10.],
       [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  9., 10.]])

In [7]:
np.exp(-k*tau_0_plus)
eta_matrix = eta_0 * np.ones([1, eta_0.shape[1]]).T
eta_matrix

array([[  0., 100., -90., 100., -90., 100., -90., 100., 100.,   0.],
       [  0., 100., -90., 100., -90., 100., -90., 100., 100.,   0.],
       [  0., 100., -90., 100., -90., 100., -90., 100., 100.,   0.],
       [  0., 100., -90., 100., -90., 100., -90., 100., 100.,   0.],
       [  0., 100., -90., 100., -90., 100., -90., 100., 100.,   0.],
       [  0., 100., -90., 100., -90., 100., -90., 100., 100.,   0.],
       [  0., 100., -90., 100., -90., 100., -90., 100., 100.,   0.],
       [  0., 100., -90., 100., -90., 100., -90., 100., 100.,   0.],
       [  0., 100., -90., 100., -90., 100., -90., 100., 100.,   0.],
       [  0., 100., -90., 100., -90., 100., -90., 100., 100.,   0.]])

In [8]:
tau_matrix_1 = np.roll(tau_matrix,-1) # numero de fila es j, numero de columna es i
tau_matrix_1

array([[ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  9., 10.,  0.],
       [ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  9., 10.,  0.],
       [ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  9., 10.,  0.],
       [ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  9., 10.,  0.],
       [ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  9., 10.,  0.],
       [ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  9., 10.,  0.],
       [ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  9., 10.,  0.],
       [ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  9., 10.,  0.],
       [ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  9., 10.,  0.],
       [ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  9., 10.,  0.]])

In [9]:
tau_matrix_diff = tau_matrix - tau_matrix.T
tau_matrix_diff

array([[  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   9.,  10.],
       [ -1.,   0.,   1.,   2.,   3.,   4.,   5.,   6.,   8.,   9.],
       [ -2.,  -1.,   0.,   1.,   2.,   3.,   4.,   5.,   7.,   8.],
       [ -3.,  -2.,  -1.,   0.,   1.,   2.,   3.,   4.,   6.,   7.],
       [ -4.,  -3.,  -2.,  -1.,   0.,   1.,   2.,   3.,   5.,   6.],
       [ -5.,  -4.,  -3.,  -2.,  -1.,   0.,   1.,   2.,   4.,   5.],
       [ -6.,  -5.,  -4.,  -3.,  -2.,  -1.,   0.,   1.,   3.,   4.],
       [ -7.,  -6.,  -5.,  -4.,  -3.,  -2.,  -1.,   0.,   2.,   3.],
       [ -9.,  -8.,  -7.,  -6.,  -5.,  -4.,  -3.,  -2.,   0.,   1.],
       [-10.,  -9.,  -8.,  -7.,  -6.,  -5.,  -4.,  -3.,  -1.,   0.]])

In [10]:
tau_matrix_diff

array([[  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   9.,  10.],
       [ -1.,   0.,   1.,   2.,   3.,   4.,   5.,   6.,   8.,   9.],
       [ -2.,  -1.,   0.,   1.,   2.,   3.,   4.,   5.,   7.,   8.],
       [ -3.,  -2.,  -1.,   0.,   1.,   2.,   3.,   4.,   6.,   7.],
       [ -4.,  -3.,  -2.,  -1.,   0.,   1.,   2.,   3.,   5.,   6.],
       [ -5.,  -4.,  -3.,  -2.,  -1.,   0.,   1.,   2.,   4.,   5.],
       [ -6.,  -5.,  -4.,  -3.,  -2.,  -1.,   0.,   1.,   3.,   4.],
       [ -7.,  -6.,  -5.,  -4.,  -3.,  -2.,  -1.,   0.,   2.,   3.],
       [ -9.,  -8.,  -7.,  -6.,  -5.,  -4.,  -3.,  -2.,   0.,   1.],
       [-10.,  -9.,  -8.,  -7.,  -6.,  -5.,  -4.,  -3.,  -1.,   0.]])

In [11]:
tau_matrix_diff = np.where(tau_matrix_diff>0, tau_matrix_diff, 0)
tau_matrix_diff

array([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  9., 10.],
       [ 0.,  0.,  1.,  2.,  3.,  4.,  5.,  6.,  8.,  9.],
       [ 0.,  0.,  0.,  1.,  2.,  3.,  4.,  5.,  7.,  8.],
       [ 0.,  0.,  0.,  0.,  1.,  2.,  3.,  4.,  6.,  7.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  2.,  3.,  5.,  6.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  2.,  4.,  5.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  3.,  4.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  2.,  3.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [12]:
arr = np.array([[1,2,3,3],[4,5,6,6]])
print(arr)
np.sum(arr, axis=0)

[[1 2 3 3]
 [4 5 6 6]]


array([5, 7, 9, 9])

In [13]:
tau_matrix_diff_1 = tau_matrix_1 - tau_matrix.T
tau_matrix_diff_1 = np.where(tau_matrix_diff_1>0, tau_matrix_diff_1, 0)
tau_matrix_diff_1


array([[ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  9., 10.,  0.],
       [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  8.,  9.,  0.],
       [ 0.,  0.,  1.,  2.,  3.,  4.,  5.,  7.,  8.,  0.],
       [ 0.,  0.,  0.,  1.,  2.,  3.,  4.,  6.,  7.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  2.,  3.,  5.,  6.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  2.,  4.,  5.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  3.,  4.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  2.,  3.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [14]:
tau_matrix_diff = tau_matrix - tau_matrix.T
tau_matrix_diff = np.where(tau_matrix_diff>0, tau_matrix_diff, 0)

tau_matrix_diff_1 = tau_matrix_1 - tau_matrix.T
tau_matrix_diff_1 = np.where(tau_matrix_diff_1>0, tau_matrix_diff_1, 0)

alpha_tau_matrix = eta_matrix * (np.exp(
    -k * tau_matrix_diff_1
) - np.exp(-k * tau_matrix_diff))

alpha_tau_matrix

array([[-0.00000000e+000, -1.38389653e-085,  1.72365264e-172,
        -2.65039655e-259, -0.00000000e+000,  0.00000000e+000,
        -0.00000000e+000,  0.00000000e+000,  0.00000000e+000,
         0.00000000e+000],
       [ 0.00000000e+000, -1.00000000e+002,  1.24550687e-085,
        -1.91516960e-172,  2.38535690e-259,  0.00000000e+000,
        -0.00000000e+000,  0.00000000e+000,  0.00000000e+000,
         0.00000000e+000],
       [ 0.00000000e+000,  0.00000000e+000,  9.00000000e+001,
        -1.38389653e-085,  1.72365264e-172, -2.65039655e-259,
        -0.00000000e+000,  0.00000000e+000,  0.00000000e+000,
         0.00000000e+000],
       [ 0.00000000e+000,  0.00000000e+000, -0.00000000e+000,
        -1.00000000e+002,  1.24550687e-085, -1.91516960e-172,
         2.38535690e-259,  0.00000000e+000,  0.00000000e+000,
         0.00000000e+000],
       [ 0.00000000e+000,  0.00000000e+000, -0.00000000e+000,
         0.00000000e+000,  9.00000000e+001, -1.38389653e-085,
         1.72365264e-172

In [15]:
alpha_tau = np.sum(alpha_tau_matrix, axis=0)
alpha_tau

array([   0., -100.,   90., -100.,   90., -100.,   90., -100., -100.,
          0.])

Alpha tau should be possitive. I don't know why are they using - 1/k. May be I should modifty it to be possitive inverting the simbols

In [16]:
alpha_s_plus = np.sum(-np.where(alpha_tau>=0,alpha_tau, 0)/k)
alpha_s_plus 

-1.35

In [17]:
alpha_s_minus = np.sum(np.where(alpha_tau<=0,alpha_tau, 0)/k)
alpha_s_minus

-2.5

In [18]:
integral_alpha_s = alpha_s_plus - alpha_s_minus
integral_alpha_s


1.15

## Calculating sum log alpha t_i

In [19]:
import numpy as np
T=10
tau_0_plus = np.array([[0.1, 0.3, 0.5,0.7,0.9]]) * T
tau_0_minus = np.array([[0.2, 0.4, 0.6]])*T
t_plus = np.array([[0.11, 0.31, 0.51, 0.71, 0.91]]) * T
t_minus = np.array([[0.21, 0.41, 0.61]]) * T
eta_plus = 100
eta_minus = 90
k = 200
theta = 1

tau_0 = np.concatenate(
    [
        tau_0_minus,
        tau_0_plus,
    ],
    axis=1,
)

eta_minus_vector = -np.ones([tau_0_minus.shape[1], 1]) * eta_minus
eta_plus_vector = np.ones([tau_0_plus.shape[1], 1]) * eta_plus
eta_vector = np.concatenate([eta_minus_vector, eta_plus_vector], axis=0).T
tau_eta = np.concatenate([tau_0, eta_vector])
tau_eta = tau_eta[:, tau_eta[0, :].argsort()]
tau_0 = tau_eta[:, tau_eta[0, :].argsort()][0:1, :]
eta_0 = tau_eta[:, tau_eta[0, :].argsort()][1:2, :]

tau_matrix = tau_0 * np.ones([1, t_plus.shape[1]]).T
eta_matrix = eta_0 * np.ones([1, t_plus.shape[1]]).T
t_plus_matrix = t_plus.T * np.ones([1, tau_0.shape[1]])  # numero de fila es t, numero de columna es tau

tau_matrix_diff = t_plus_matrix - tau_matrix 
tau_matrix_diff = np.where(tau_matrix_diff > 0, tau_matrix_diff, 0)

alpha_tau_matrix = eta_matrix * (np.exp(-k * tau_matrix_diff))
alpha_tau = np.sum(alpha_tau_matrix, axis=1)
# The signs seem to be wrong in the paper, keeping them anyway
sum_log_alpha_plus = np.sum(np.log(np.where(alpha_tau >= 0, alpha_tau, 0) + theta))
sum_log_alpha_plus

18.995638800241302

In [20]:
np.log(1)

0.0

## Generating simulation data to estimate parameters


In [1]:
from simulations import generate_simulations_bunchs
from parameters import Parameters, base_simulation_parameters_dict
import pandas as pd
import numpy as np


p = Parameters(**base_simulation_parameters_dict)
p.dalpha = 30
print(p)
results = generate_simulations_bunchs(p)


Parameters(q_max=4, T=60, A=300, dalpha=30, Delta=0.005, epsilon=0.005, psi=0.01, phi_=1e-06, eta_plus=60.0, eta_minus=60.0, sigma=0.01, k=200.0, xi=1.0, lambda_plus=1.0, lambda_minus=1.0, theta=0.1, s0=100, n=200, drift=True)
Calculating h
Starting h calculation
0.0004995003608890108
120120
Starting to calculate h
Finished calculating h0.0 h                          e-07 h                            
Amount of simulations: 200
Mean of PNL:0.2342749999998560253993e-07 h                              
Stde of PNL:0.18778471549898396
500


In [44]:
l = 200

## Option A: Flattening of times


In [4]:
tau0_plus = np.array([
    np.array(np.where(results["dMt0_plus"][:l,:])[1]
                               ) .flatten()
                      *p.dt]
                      )
tau0_minus = np.array([
    np.array(np.where(results["dMt0_minus"][:l,:])[1]
                                ) .flatten()
                       *p.dt]
)

print("Tau 0 minus ", tau0_minus)
print("Tau 0 plus ", tau0_plus)

Tau 0 minus  [[ 1.08291678  2.37362571  4.9890096   5.1208777   6.90709099  6.97552254
   7.72277508  8.22177594  8.97452298 12.28421238 13.18680953 13.80668948
  13.81218398 16.06443111 18.51298188 19.09140329 22.11387998 22.38710667
  24.79819492 25.76972312 26.59689572 27.86662563 28.98051144 30.18180981
  30.63235913 31.18580553 31.98900211 32.32666436 32.50948149 34.07291762
  34.48850192 34.51097943 35.4370531  36.30718273 37.55893064 39.37411495
  40.89359505 43.96502276 44.74174583 46.81966733 47.34713971 48.09489175
  49.35712916 49.4135727  49.57840782 51.15433146 52.1808047  52.44104439
  52.49748793 53.06541984 53.31067452 53.61836674 54.47401086 55.356628
  56.49698732 56.52695734 56.53544885 56.76721701  0.19730264  0.81618359
   1.72677275  2.19730209  2.75124799  3.22327583  3.94455435  4.06443444
   4.13436449  6.57442375  6.67432382  8.33366402  8.47352412  8.65134625
   8.91757994  9.86113612 10.05344376 12.25873786 13.142354   14.16732874
  15.0799159  16.16832718 1

In [5]:
tau0_minus.shape

(1, 290)

In [7]:
s_jumps = (results['s']-np.roll(results['s'], 1))
t_minus = np.array([(np.array(np.where(s_jumps[:l,:] < 0)[1]).flatten() 
                     * p.dt)])
t_plus = np.array([np.array(np.where(s_jumps[:l,:] > 0)[1]).flatten() 
                   * p.dt])
print("t_minus", t_minus)
print("t_plus", t_plus)
print("s_jumps", s_jumps)

t_minus [[ 1.09190779  1.85814134  6.91308499  7.72727058  8.22427344  8.22976795
  16.06842711 22.38910468 22.39110268 24.5004927  24.80318992 26.59739522
  27.05194055 27.5469454  27.87561664 32.51397699 34.49249792 34.51197843
  36.31467524 37.57541415 40.90758106 44.1478399  49.57990632 51.15582996
  53.06591934 53.22376145 53.31117402 56.52895534  0.20529465  1.61738217
   1.78671279  8.33416352  8.33865902  8.48301463 10.16583134 12.25973686
  13.1438525  19.19829637 19.26073392 21.98450938 24.32666658 29.56093086
  29.56942236 29.9010906  31.87311803 35.8511389  36.5374524  38.22576362
  39.20728183 40.15483401 40.15733151 41.61886957 43.86512269 44.51247516
  46.85113585 50.3836024  51.72326237 54.34513976  2.02547396  2.39060873
  14.93006579 16.16782768 18.12287209 22.94754608 31.66282838 36.59889094
  39.60888012 40.60987884 43.81816966 46.40208503 46.79119581 47.40957725
  50.94753831 51.26272304 51.39159413 52.48400142  0.          1.43706254
   4.6992994   4.89060803  4.9

In [91]:
m.likelihood_to_minimize(x0)

-6419.649675058034

In [90]:
import numpy as np
from scipy.optimize import minimize
from scipy import optimize
from likelihood import MaximumLikelihood

m = MaximumLikelihood(
    p.T,
    tau0_plus,
    tau0_minus,
    t_plus,
    t_minus
)
# def multi_objective_function(x):
#     return [objective_function1(x), objective_function2(x)]
x0 = np.array([10, 100, 100, 1])  # the 4 parameters to optimize
bounds = [(1, 500), (1, 200),(1, 200),(0, 1)]
res = minimize(m.likelihood_to_minimize, x0, method='BFGS',#
               #bounds=bounds,
               options={'disp': True, 
                        # 'maxiter':1000, "maxfun":2000
                        },
               )
# res = optimize.fmin_cobyla(m.likelihood_to_minimize, x0, cons=[], args=(), maxfun=100000,)
# optimize.differential_evolution(m.likelihood_to_minimize, bounds, args=(), strategy='rand1exp', maxiter=10000, popsize=15, tol=0.01, mutation=(0.5, 1), recombination=0.7, seed=None, callback=None, disp=False, polish=True, init='latinhypercube', atol=0, updating='immediate', workers=1, constraints=(), x0=None,)

print(res)

         Current function value: -6419.649675
         Iterations: 0
         Function evaluations: 390
         Gradient evaluations: 77
  message: Desired error not necessarily achieved due to precision loss.
  success: False
   status: 2
      fun: -6419.649675058034
        x: [ 1.000e+01  1.000e+02  1.000e+02  1.000e+00]
      nit: 0
      jac: [ 5.744e+02 -3.170e+00 -5.554e+01  1.963e+01]
 hess_inv: [[1 0 0 0]
            [0 1 0 0]
            [0 0 1 0]
            [0 0 0 1]]
     nfev: 390
     njev: 77


/Users/federico/miniconda3/envs/marketmaking/lib/python3.11/site-packages/scipy/optimize/_minimize.py:705: OptimizeWarning: Desired error not necessarily achieved due to precision loss.
  res = _minimize_bfgs(fun, x0, args, jac, callback, **options)


## Option B: Not flattening of data but rather optimizing for the sum of results

In [45]:
from likelihood import MaximumLikelihood
tau0_plus_dict = MaximumLikelihood.get_times(zip(*np.where(results["dMt0_plus"][:l,:])), p.dt)
tau0_minus_dict = MaximumLikelihood.get_times(zip(*np.where(results["dMt0_minus"][:l,:])), p.dt)
tau0_minus_dict = MaximumLikelihood.get_times(zip(*np.where(results["dMt0_minus"][:l,:])), p.dt)
s_jumps = (results['s']-np.roll(results['s'], 1))
t_minus_dict = MaximumLikelihood.get_times(zip(*np.where(s_jumps[:l,:] < 0)), p.dt)
t_plus_dict = MaximumLikelihood.get_times(zip(*np.where(s_jumps[:l,:] > 0)), p.dt)
t_plus_dict

{0: [0.0,
  10.312184950553627,
  18.38011477963293,
  20.366627714888526,
  23.93056278983162,
  26.053439323609915,
  28.777214791537688,
  39.82516377368083,
  50.749736166684386,
  52.91606923186002,
  54.03195303808607,
  54.77820657725426,
  54.86761714185339,
  58.324659139566236,
  59.983999338439524],
 1: [0.0,
  9.163334120508903,
  11.648348415931732,
  11.922574114059799,
  14.4360599300533,
  16.605889497755165,
  17.947047966742158,
  22.325168629934335,
  22.860633016807355,
  28.673318716472775,
  34.16682368553012,
  34.89759271351074,
  35.917072950085206,
  39.29719189222114,
  45.59239494050535,
  47.57591087359561,
  48.083902740619735,
  48.8081782639088,
  55.44304155759753],
 2: [0.0,
  0.46653333707033606,
  4.404594182319297,
  4.4085901852064096,
  4.601896824870456,
  4.60539332739668,
  7.253744240830215,
  11.461535280959241,
  19.45953505951408,
  20.57741686718369,
  25.79270013522585,
  27.62087145607963,
  33.61787278891309,
  34.077413120930984,
  35.

In [46]:
multi_params = []
for i in range(l):
    multi_params.append(
        {
            "T": p.T,
            "tau_0_plus": np.array([tau0_plus_dict[i]]),
            "tau_0_minus": np.array([tau0_minus_dict[i]]),
            "t_plus": np.array([t_plus_dict[i]]),
            "t_minus": np.array([t_minus_dict[i]])
            })
multi_params[0].keys()

dict_keys(['T', 'tau_0_plus', 'tau_0_minus', 't_plus', 't_minus'])

In [47]:
multi_params[0]['tau_0_minus']

array([[ 1.08291678,  2.37362571,  4.9890096 ,  5.1208777 ,  6.90709099,
         6.97552254,  7.72277508,  8.22177594,  8.97452298, 12.28421238,
        13.18680953, 13.80668948, 13.81218398, 16.06443111, 18.51298188,
        19.09140329, 22.11387998, 22.38710667, 24.79819492, 25.76972312,
        26.59689572, 27.86662563, 28.98051144, 30.18180981, 30.63235913,
        31.18580553, 31.98900211, 32.32666436, 32.50948149, 34.07291762,
        34.48850192, 34.51097943, 35.4370531 , 36.30718273, 37.55893064,
        39.37411495, 40.89359505, 43.96502276, 44.74174583, 46.81966733,
        47.34713971, 48.09489175, 49.35712916, 49.4135727 , 49.57840782,
        51.15433146, 52.1808047 , 52.44104439, 52.49748793, 53.06541984,
        53.31067452, 53.61836674, 54.47401086, 55.356628  , 56.49698732,
        56.52695734, 56.53544885, 56.76721701]])

In [50]:
import numpy as np
from scipy.optimize import minimize
from scipy import optimize
from likelihood import MaximumLikelihood

def multi_likelihood_to_minimize(x):
    m = MaximumLikelihood.multi_likelihood_to_minimize(multi_params, x)/len(multi_params)/10000000
    print(m)
    return m
x0 = np.array([10, 100, 100, 1])  # the 4 parameters to optimize
x0 = np.random.rand(4)*100
bounds = [(1, 500), (1, 200),(1, 200),(0, 1)]
res = minimize(multi_likelihood_to_minimize, x0, #method='CG',#
            #    bounds=bounds,
               #options={'disp': True, 
                        # 'maxiter':1000, "maxfun":2000
               #         },
               )
# res = optimize.fmin_cobyla(m.likelihood_to_minimize, x0, cons=[], args=(), maxfun=100000,)
# optimize.differential_evolution(m.likelihood_to_minimize, bounds, args=(), strategy='rand1exp', maxiter=10000, popsize=15, tol=0.01, mutation=(0.5, 1), recombination=0.7, seed=None, callback=None, disp=False, polish=True, init='latinhypercube', atol=0, updating='immediate', workers=1, constraints=(), x0=None,)

print(res)

/Users/federico/Repos/marketmaking/alpha_signals/likelihood.py:103: RuntimeWarning: invalid value encountered in log
  result = np.sum(np.log(np.where(alpha_tau >= 0, alpha_tau, 0) + theta))
/Users/federico/Repos/marketmaking/alpha_signals/likelihood.py:137: RuntimeWarning: invalid value encountered in log
  result = np.sum(np.log(np.where(alpha_tau <= 0, -alpha_tau, 0) + theta))
/Users/federico/Repos/marketmaking/alpha_signals/likelihood.py:77: RuntimeWarning: overflow encountered in exp
  np.exp(-k * tau_matrix_diff_1) - np.exp(-k * tau_matrix_diff)
/Users/federico/Repos/marketmaking/alpha_signals/likelihood.py:77: RuntimeWarning: invalid value encountered in subtract
  np.exp(-k * tau_matrix_diff_1) - np.exp(-k * tau_matrix_diff)
/Users/federico/Repos/marketmaking/alpha_signals/likelihood.py:76: RuntimeWarning: overflow encountered in multiply
  alpha_tau_matrix = eta_matrix * (
/Users/federico/Repos/marketmaking/alpha_signals/likelihood.py:76: RuntimeWarning: invalid value encounte

  message: Desired error not necessarily achieved due to precision loss.
  success: False
   status: 2
      fun: nan
        x: [ 2.032e+01  7.311e+00  6.662e+01 -1.562e+02]
      nit: 2
      jac: [       nan        nan        nan        nan]
 hess_inv: [[ 1.716e+05 -6.292e+04 -9.250e+04  4.646e+06]
            [-6.292e+04  2.307e+04  3.392e+04 -1.704e+06]
            [-9.250e+04  3.392e+04  4.986e+04 -2.504e+06]
            [ 4.646e+06 -1.704e+06 -2.504e+06  1.258e+08]]
     nfev: 1115
     njev: 223


## Getting real data to estimate brazil parameters

In [ ]:
import pandas as pd
df = pd.read_csv("brazil_parameters/md_20230731.csv")
df["timestamp"] = pd.to_datetime(df["timestamp"])

In [1]:
df["timestamp"] =  df["timestamp"] - df["timestamp"].min()
df["time"] = (df['timestamp'] - df['timestamp'].min()).dt.total_seconds()

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [ ]:
tau0_plus = df[df["ismoplus"]]['time'].unique()
tau0_minus = df[df["ismominus"]]['time'].unique()
t_minus = df[df["isjumpplus"]]['time'].unique()
t_plus = df[df["isjumpminus"]]['time'].unique()
T=df["time"].max()